In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import plotly.express as px
from shapely.geometry import Point

# Load petrol stations and probe data
petrol_stations = gpd.read_file(r"D:\HERE INDIA HACKATHON 2025\Stale_Petrol_station\Petrol_Station\Petrol_Station\petrol_station_jakarta.shp")
probe_data = gpd.read_file(r"D:\HERE INDIA HACKATHON 2025\Stale_Petrol_station\IDN_Q223_May23_Jakarta\Probe.shp")

# Load updated CSV with fclass and tunnel information
data = pd.read_csv(r"D:\HERE INDIA HACKATHON 2025\probes_with_stations_and_roads.csv")

# Ensure latitude & longitude exist before creating geometry
if "displaylon" in data.columns and "displaylat" in data.columns:
    data["geometry"] = data.apply(lambda row: Point(row["displaylon"], row["displaylat"]), axis=1)
else:
    raise ValueError("Longitude and Latitude columns are missing in the CSV file.")

# Convert DataFrame to GeoDataFrame with WGS 84 CRS (EPSG:4326)
data = gpd.GeoDataFrame(data, geometry="geometry", crs="EPSG:4326")
probe_data = probe_data.to_crs("EPSG:4326")
petrol_stations = petrol_stations.to_crs("EPSG:4326")

# Rule-Based Detection: Low Traffic & High Distance
weight_threshold = data["WEIGHT"].quantile(0.25)  # Bottom 25% in traffic
dist_threshold = data["distance_to_station"].quantile(0.75)  # Top 25% in distance

# Penalize based on fclass (minor roads) and tunnel (in tunnels more likely stale)
minor_road_classes = ["residential", "unclassified", "living_street"]
data["road_penalty"] = data["fclass"].apply(lambda x: 1 if x in minor_road_classes else 0)
data["tunnel_penalty"] = data["tunnel"].apply(lambda x: 1 if x == "T" else 0)

rule_based_stale = data[
    (data["WEIGHT"] <= weight_threshold) &
    (data["distance_to_station"] >= dist_threshold) &
    ((data["road_penalty"] == 1) | (data["tunnel_penalty"] == 1))
]

# Outlier-Based Detection: IQR Method on Probe Count
projected_crs = "EPSG:32748"  # UTM Zone for Jakarta
petrol_stations_projected = petrol_stations.to_crs(projected_crs)
probe_data_projected = probe_data.to_crs(projected_crs)

# Adaptive Buffer Calculation (Dynamic Buffering Based on Probe Density)
baseline_buffer = 100
petrol_stations_projected["baseline_buffer"] = petrol_stations_projected.geometry.buffer(baseline_buffer)

# Perform spatial join to count probes within the buffer
buffered_stations = gpd.GeoDataFrame(petrol_stations_projected, geometry="baseline_buffer", crs=projected_crs)
merged_data = gpd.sjoin(probe_data_projected, buffered_stations, how="inner", predicate="within")

# Count probes near each petrol station
probe_counts = merged_data.groupby("placeid").size().reset_index(name="probe_count")
petrol_stations_projected = petrol_stations_projected.merge(probe_counts, on="placeid", how="left").fillna(0)

# Adaptive Buffer Calculation
mean_probe_density = petrol_stations_projected["probe_count"].mean()

def calculate_adaptive_buffer(probe_count, baseline_buffer, mean_density):
    return baseline_buffer * (1 + (mean_density / (probe_count + 1)))

petrol_stations_projected["adaptive_buffer"] = petrol_stations_projected["probe_count"].apply(
    lambda count: calculate_adaptive_buffer(count, baseline_buffer, mean_probe_density)
)

# Apply adaptive buffers
petrol_stations_projected["buffer"] = petrol_stations_projected.geometry.buffer(petrol_stations_projected["adaptive_buffer"])

# Perform spatial join with adaptive buffers
buffered_stations = gpd.GeoDataFrame(petrol_stations_projected, geometry="buffer", crs=projected_crs)
merged_data = gpd.sjoin(probe_data_projected, buffered_stations, how="inner", predicate="within")
probe_counts = merged_data.groupby("placeid").size().reset_index(name="probe_count")

# Merge updated probe counts back to petrol stations
petrol_stations_projected = petrol_stations_projected.drop(columns="probe_count").merge(probe_counts, on="placeid", how="left").fillna(0)

# Compute IQR for adaptive probe counts
Q1 = petrol_stations_projected["probe_count"].quantile(0.25)
Q3 = petrol_stations_projected["probe_count"].quantile(0.75)
IQR = Q3 - Q1

# Detect outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outlier_stale = petrol_stations_projected[
    (petrol_stations_projected["probe_count"] < lower_bound) | 
    (petrol_stations_projected["probe_count"] > upper_bound)
]

# Convert outlier_stale back to EPSG:4326 before merging
outlier_stale = outlier_stale.to_crs("EPSG:4326")

# Merge rule-based and outlier stale stations
rule_based_stale = rule_based_stale.drop(columns=["geometry"], errors="ignore")
outlier_stale = outlier_stale.drop(columns=["baseline_buffer", "buffer"], errors="ignore")

stale_stations = pd.concat([rule_based_stale, outlier_stale], ignore_index=True).drop_duplicates()
stale_stations = gpd.GeoDataFrame(stale_stations, geometry="geometry", crs="EPSG:4326")

# Identify probes near stale stations
buffered_stale_stations = stale_stations.to_crs(projected_crs).geometry.buffer(100)
buffered_stale_stations = gpd.GeoDataFrame(geometry=buffered_stale_stations, crs=projected_crs)
probes_near_stale = gpd.sjoin(probe_data_projected, buffered_stale_stations, how="inner", predicate="within")
probes_near_stale = probes_near_stale.to_crs("EPSG:4326")

# Visualize with Plotly
plot_data = pd.DataFrame({
    "lon": pd.concat([petrol_stations.geometry.x, stale_stations.geometry.x, probes_near_stale.geometry.x]),
    "lat": pd.concat([petrol_stations.geometry.y, stale_stations.geometry.y, probes_near_stale.geometry.y]),
    "type": ["Active Station"] * len(petrol_stations) +
            ["Stale Station"] * len(stale_stations) +
            ["Probes Near Stale"] * len(probes_near_stale)
})

fig = px.scatter_mapbox(
    plot_data,
    lat="lat",
    lon="lon",
    color="type",
    zoom=12,
    mapbox_style="carto-positron",
    title="Stale Petrol Stations Detection (Active + Stale + Probe Visualization)"
)

fig.show()